# **Assignment 1, classification task**
### Ostapovich Oleg

### Data preprocessing

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from functions import *
import matplotlib.pyplot as plt

In [2]:
#loading data
stream_quality_df = pd.read_csv('stream_quality_data/train_data.csv')
stream_quality_test = pd.read_csv('stream_quality_data/test_data.csv')

In [3]:
stream_quality_df.head()

,fps_mean,fps_std,fps_lags,rtt_mean,rtt_std,dropped_frames_mean,dropped_frames_std,dropped_frames_max,auto_bitrate_state,auto_fec_state,auto_fec_mean,stream_quality
0,24.4,0.516398,0,91.1,6.723921,0.0,0.0,0.0,off,partial,50.0,0
1,28.6,2.065591,0,99.7,15.923777,0.0,0.0,0.0,off,partial,50.0,0
2,30.0,0.000000,0,98.1,11.798776,0.0,0.0,0.0,off,partial,50.0,0
3,30.3,0.948683,0,99.4,13.014522,0.0,0.0,0.0,off,partial,50.0,0
4,29.9,0.316228,0,123.2,62.476307,0.0,0.0,0.0,off,partial,50.0,0


In [4]:
# Data preprocessing at labels
stream_quality_df, stream_quality_test = LabelEncoderforClassification(stream_quality_df,stream_quality_test)

In [5]:
stream_quality_df.head() #here what happened with data

,fps_mean,fps_std,fps_lags,rtt_mean,rtt_std,dropped_frames_mean,dropped_frames_std,dropped_frames_max,auto_bitrate_state,auto_fec_state,auto_fec_mean,stream_quality
0,24.4,0.516398,0,91.1,6.723921,0.0,0.0,0.0,1,1,50.0,0
1,28.6,2.065591,0,99.7,15.923777,0.0,0.0,0.0,1,1,50.0,0
2,30.0,0.000000,0,98.1,11.798776,0.0,0.0,0.0,1,1,50.0,0
3,30.3,0.948683,0,99.4,13.014522,0.0,0.0,0.0,1,1,50.0,0
4,29.9,0.316228,0,123.2,62.476307,0.0,0.0,0.0,1,1,50.0,0


In [6]:
# Data preprocessing at numerical data
stream_quality_df, stream_quality_test = NumericalEncoderforClassification(stream_quality_df,stream_quality_test)

In [7]:
stream_quality_df.head() #here what happened with data

,fps_mean,fps_std,fps_lags,rtt_mean,rtt_std,dropped_frames_mean,dropped_frames_std,dropped_frames_max,auto_bitrate_state,auto_fec_state,auto_fec_mean,stream_quality
0,0.191975,0.001652,0.0,0.007063,0.000165,0.0,0.0,0.0,1,1,0.2,0
1,0.225020,0.006609,0.0,0.007730,0.000391,0.0,0.0,0.0,1,1,0.2,0
2,0.236035,0.000000,0.0,0.007606,0.000290,0.0,0.0,0.0,1,1,0.2,0
3,0.238395,0.003035,0.0,0.007706,0.000320,0.0,0.0,0.0,1,1,0.2,0
4,0.235248,0.001012,0.0,0.009552,0.001534,0.0,0.0,0.0,1,1,0.2,0


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchmetrics import F1Score, Accuracy
import time
from torch.utils.tensorboard import SummaryWriter

In [9]:
from sklearn.ensemble import IsolationForest

iforest = IsolationForest()
X = stream_quality_df.iloc[:, :8]
iforest.fit(X)
pred = iforest.predict(X)
outliers = stream_quality_df[pred == -1]
stream_quality_df_valid = stream_quality_df[pred != -1]

print("Original Samples : ",stream_quality_df.shape)
print("Number of Outliers : ", outliers.shape)
print("Number of Normal Samples : ", stream_quality_df_valid.shape)

Original Samples :  (406572, 12)
Number of Outliers :  (36759, 12)
Number of Normal Samples :  (369813, 12)


In [10]:
y_train_df = stream_quality_df_valid['stream_quality']
x_train_df = stream_quality_df_valid.drop(['stream_quality'], axis=1)

y_test = stream_quality_test['stream_quality']
x_test = stream_quality_test.drop(['stream_quality'], axis=1)

In [17]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
x_train_resampled, y_train_resampled = smote.fit_resample(x_train_df,y_train_df)
print('X_train shape:', x_train_df.shape)
print('X_train resampled shape:', x_train_resampled.shape)

X_train shape: (369813, 11)
X_train resampled shape: (710236, 11)


In [18]:
print("before split", x_train_df.shape, y_train_df.shape, x_test.shape, y_test.shape)
x_train, x_val, y_train, y_val = train_test_split(x_train_resampled, y_train_resampled, train_size=0.8)
print("after split", x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape)

before split (369813, 11) (369813,) (243596, 11) (243596,)
after split (568188, 11) (568188,) (142048, 11) (142048,) (243596, 11) (243596,)


In [19]:
class Dset(Dataset):
    def __init__(self, values, labels):
        self.x = torch.tensor(values.values, dtype= torch.float32)
        # self.y = torch.from_numpy(labels.values).type(torch.LongTensor)
        self.y = list(map(int, labels.values))
    def __len__(self):
        return (len(self.y))

    def __getitem__(self, i):
        return self.x[i], self.y[i]

In [20]:
batch_size = 100
# Adding data to loader
trainloader = torch.utils.data.DataLoader(Dset(x_train,y_train), batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=0
                                          )
valloader = torch.utils.data.DataLoader(Dset(x_val,y_val), batch_size=batch_size,
                                          shuffle=True,
                                        num_workers=0
                                        )
testloader = torch.utils.data.DataLoader(Dset(x_test,y_test), batch_size=batch_size,
                                          shuffle=True,
                                         num_workers=0
                                         )

In [21]:
def train_evaluate(net, optimizer, writer, epochs):
    starttime = time.time()
    criterion = nn.CrossEntropyLoss()
    # criterion = nn.NLLLoss()
    accuracy_func = Accuracy(num_classes=3, average='weighted').to(device)
    f1_score_func = F1Score(num_classes=3, average='weighted').to(device)
    for epoch in range(epochs):

        running_loss = 0.0
        f1 = 0
        accuracy = 0
        for data in trainloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            _, pred = torch.max(outputs.data, 1)
            accuracy += accuracy_func(pred, labels) * 100
            f1 += f1_score_func(pred, labels) * 100

        running_loss /= len(trainloader)
        accuracy /= len(trainloader)
        f1 /= len(trainloader)
        writer.add_scalar('Training_Loss', running_loss, epoch)
        writer.add_scalar('Training_Accuracy', accuracy, epoch)
        writer.add_scalar('Training_F1', f1, epoch)

        print('Epoch {} - train loss:{}, accuracy:{}, f1_score:{}, time passed {}s'.format(epoch+1, running_loss, accuracy, f1, int(time.time()-starttime)))

        val_loss = 0.0
        val_accuracy = 0
        val_f1_score = 0
        with torch.no_grad():
            for data in valloader:
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                _, pred = torch.max(outputs.data, 1)
                val_loss += loss.item()
                val_accuracy += accuracy_func(pred, labels) * 100
                val_f1_score += f1_score_func(pred, labels) * 100
            val_loss /= len(valloader)
            val_accuracy /= len(valloader)
            val_f1_score /= len(valloader)
            writer.add_scalar('Val_Loss', val_loss, epoch)
            writer.add_scalar('Val_Accuracy', val_accuracy, epoch)
            writer.add_scalar('Val_F1', val_f1_score, epoch)

        print('Epoch {} - val loss:{}, accuracy:{}, f1_score:{}, time passed {}s'.format(epoch+1, val_loss, val_accuracy, val_f1_score, int(time.time()-starttime)))

In [22]:
def test_evaluate(net, optimizer):
    val_loss = 0.0
    val_accuracy = 0
    val_f1_score = 0
    criterion = nn.CrossEntropyLoss()
    accuracy_func = Accuracy(num_classes=3, average='weighted').to(device)
    f1_score_func = F1Score(num_classes=3, average='weighted').to(device)
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            _, pred = torch.max(outputs.data, 1)
            val_loss += loss.item()
            val_accuracy += accuracy_func(pred, labels) * 100
            val_f1_score += f1_score_func(pred, labels) * 100
        val_loss /= len(testloader)
        val_accuracy /= len(testloader)
        val_f1_score /= len(testloader)
    print('Test evaluation - test loss:{}, accuracy:{}, f1_score:{}'.format(val_loss, val_accuracy, val_f1_score))

In [23]:
from torchsummary import summary
import torch
import torch.nn as nn
device = torch.device("cuda:0")

In [24]:
class StartModel(nn.Module):
    def __init__(self):
        super(StartModel, self).__init__()
        self.network=nn.Sequential(
            nn.Linear(in_features=11, out_features=3, bias=True))

    def forward(self, x):
        return self.network(x)

startmodel = StartModel().to(device)

In [26]:
optimizer = torch.optim.Adadelta(startmodel.parameters())
writer = SummaryWriter('runs1/BaseModelbase2')
train_evaluate(startmodel, optimizer, writer, 15)

Epoch 1 - train loss:0.6608105957487107, accuracy:61.020938873291016, f1_score:60.53476333618164, time passed 58s
Epoch 1 - val loss:0.6611437576493271, accuracy:61.373268127441406, f1_score:61.25039291381836, time passed 70s
Epoch 2 - train loss:0.6598881250093286, accuracy:61.08101272583008, f1_score:60.627647399902344, time passed 131s
Epoch 2 - val loss:0.660210203025477, accuracy:60.94633865356445, f1_score:60.3121452331543, time passed 143s
Epoch 3 - train loss:0.6591256231394758, accuracy:61.200199127197266, f1_score:60.757232666015625, time passed 199s
Epoch 3 - val loss:0.6595481897977605, accuracy:61.32676696777344, f1_score:61.132022857666016, time passed 212s
Epoch 4 - train loss:0.6585220160072264, accuracy:61.281028747558594, f1_score:60.8638916015625, time passed 268s
Epoch 4 - val loss:0.6589222196791056, accuracy:61.349693298339844, f1_score:61.1363410949707, time passed 281s
Epoch 5 - train loss:0.6580171445774722, accuracy:61.29631042480469, f1_score:60.8805770874023

KeyboardInterrupt: 

### Data visualization

In [ ]:
from sklearn.decomposition import PCA
# Visualizing data with PCA
dim_reducer = PCA(n_components=2)
reduced_df = dim_reducer.fit_transform(stream_quality_df)
plt.scatter(reduced_df[:, 0], reduced_df[:, 1], marker='.')
plt.show()

In [40]:
# Splitting data to X and Y
Y_train = stream_quality_df['stream_quality']
X_train = stream_quality_df.drop(['stream_quality'], axis=1)

Y_test = stream_quality_test['stream_quality']
X_test = stream_quality_test.drop(['stream_quality'], axis=1)

### Feature selection with Lasso

In [41]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.1).fit(X_train,Y_train)
y_pred = lasso.predict(X_test)
print("MSE on testset:", mean_squared_error(Y_test, y_pred))

MSE on testset: 0.06029186327204236


### Classification task

In [42]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression(penalty='l2').fit(X_train,Y_train)
y_pred = logistic.predict(X_test)
y_pred_train = logistic.predict(X_train)
print('Accuracy on testset', accuracy_score(Y_test, y_pred))
print('Precision on testset', precision_score(Y_test, y_pred))
print('Recall on testset', recall_score(Y_test,y_pred))
print('\nAccuracy on trainset', accuracy_score(Y_train, y_pred_train))
print('Precision on trainset', precision_score(Y_train, y_pred_train))
print('Recall on trainset', recall_score(Y_train,y_pred_train))

Accuracy on testset 0.9367969917404226
Precision on testset 0.5249247239879559
Recall on testset 0.1999490251051357

Accuracy on trainset 0.9447748492271971
Precision on trainset 0.8499154636493692
Recall on trainset 0.23478479557375873


In [43]:
from sklearn.linear_model import RidgeClassifier
ridge = RidgeClassifier().fit(X_train,Y_train)
y_pred = ridge.predict(X_test)
y_pred_train = ridge.predict(X_train)
print('Accuracy on testset', accuracy_score(Y_test, y_pred))
print('Precision on testset', precision_score(Y_test, y_pred))
print('Recall on testset', recall_score(Y_test,y_pred))
print('\nAccuracy on trainset', accuracy_score(Y_train, y_pred_train))
print('Precision on trainset', precision_score(Y_train, y_pred_train))
print('Recall on trainset', recall_score(Y_train,y_pred_train))

Accuracy on testset 0.9393627153155224
Precision on testset 0.6274509803921569
Recall on testset 0.14476870141455334

Accuracy on trainset 0.9436434383085899
Precision on trainset 0.9247367512515104
Recall on trainset 0.1924624559890781


RidgeClassifier and LogisticRegression give us similar results. Models work well.

In [44]:
#Let's see the results of cross validation score
from sklearn.model_selection import cross_val_score
print('Lasso cross validation score:', cross_val_score(lasso, X_test,Y_test, cv=3))
print('Logistic Regression cross validation score:', cross_val_score(logistic, X_test,Y_test, cv=3))
print('Ridge Classifier cross validation score:', cross_val_score(ridge, X_test,Y_test, cv=3))

Lasso cross validation score: [-0.00545202 -0.00371694 -0.0120003 ]
Logistic Regression cross validation score: [0.93998695 0.9407505  0.93874233]
Ridge Classifier cross validation score: [0.93938349 0.93977758 0.9385576 ]


### Removal of outliers

In [45]:
#loading data
stream_quality_df = pd.read_csv('stream_quality_data/train_data.csv')
stream_quality_test = pd.read_csv('stream_quality_data/test_data.csv')

In [48]:
# data preprocessing
stream_quality_df, stream_quality_test = LabelEncoderforClassification(stream_quality_df,stream_quality_test)
stream_quality_df, stream_quality_test = NumericalEncoderforClassification(stream_quality_df, stream_quality_test)
Y_train = stream_quality_df['stream_quality']
X_train = stream_quality_df.drop(['stream_quality'], axis=1)
Y_test = stream_quality_test['stream_quality']
X_test = stream_quality_test.drop(['stream_quality'], axis=1)

In [49]:
LogisticRegressionFunction(X_train, Y_train, X_test, Y_test)


LogisticRegression
Accuracy on testset 0.9367969917404226
Precision on testset 0.5249247239879559
Recall on testset 0.1999490251051357

Accuracy on trainset 0.9447748492271971
Precision on trainset 0.8499154636493692
Recall on trainset 0.23478479557375873


In [50]:
RidgeClassifierFunction(X_train, Y_train, X_test, Y_test)


RidgeClassifier
Accuracy on testset 0.9393627153155224
Precision on testset 0.6274509803921569
Recall on testset 0.14476870141455334

Accuracy on trainset 0.9436434383085899
Precision on trainset 0.9247367512515104
Recall on trainset 0.1924624559890781


### Balancing of Data

### Classification task with balanced and outliers free data

In [52]:
LogisticRegressionFunction(X_train_resampled, Y_train_resampled, X_test, Y_test)


LogisticRegression
Accuracy on testset 0.8512413996945762
Precision on testset 0.23000289146492128
Recall on testset 0.5575379125780553

Accuracy on trainset 0.7222921914357683
Precision on trainset 0.862835348115586
Recall on trainset 0.5286187285141708


In [53]:
RidgeClassifierFunction(X_train_resampled, Y_train_resampled, X_test, Y_test)


RidgeClassifier
Accuracy on testset 0.8570707236572029
Precision on testset 0.2367500894799152
Recall on testset 0.5479164011724226

Accuracy on trainset 0.7237800273540019
Precision on trainset 0.859626939135748
Recall on trainset 0.5349080366902714
